# 인터페이스가 간단하면 클래스 대신 함수를 받자

Python 내장 API의 상당수에는 함수를 넘겨서 동작을 사용자화하는 기능이 있다. API는 이런 hook를 이용해서 코드를 실행 중에 호출한다.

In [1]:
names = ['Socrates', 'Archimedes', 'Plato', 'Aristotle']
names.sort(key=lambda x: len(x))
print(names)

['Plato', 'Socrates', 'Aristotle', 'Archimedes']


Python의 후크 중 상당수는 인수와 반환 값을 잘 정의해놓은 단순히 상태가 없는 함수다. 함수는 클래스보다 설명하기 쉽고 정의하기도 간단해서 후크로 쓰기에 이상적이다. 함수가 후크로 동작하는 이유는 Python이 first-class function을 갖췄기 때문이다. 즉, 언어에서 함수와 method를 다른 값처럼 전달하고 참조할 수 있기 때문이다.

In [2]:
def log_missing():
    print('Key added')
    return 0

In [6]:
from collections import defaultdict

current = {'green': 12, 'blue': 3}
increments = [
    ('red', 5),
    ('blue', 17),
    ('orange', 9),
]
result = defaultdict(log_missing, current)
print('Before:', dict(result))
for key, amount in increments:
    result[key] += amount
print('After:', dict(result))

Before: {'green': 12, 'blue': 3}
Key added
Key added
After: {'green': 12, 'blue': 20, 'red': 5, 'orange': 9}


log_missing 같은 함수를 넘기면 결정 동작과 부작용을 분리하므로 API를 쉽게 구축하고 테스트 할 수 있다.

다음은 상태 보존 closure를 기본값 후크로 사용하는 헬퍼 함수다.

In [7]:
def increment_with_report(current, increments):
    added_count = 0
    
    def missing():
        nonlocal added_count # 상태 보존 closure
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
        
    return result, added_count

defaultdict는 missing 후크가 상태를 유지한다는 사실을 모르지만 increment_with_report 함수를 실행하면 tuple의 요소로 기대한 개수인 2를 얻는다.  
__이는 간단한 함수를 인터페이스용으로 사용할 때 얻을 수 있는 또 다른 이점이다. closure 안에 상태를 숨기면 나중에 기능을 추가하기도 쉽다.__

In [8]:
result, count = increment_with_report(current, increments)
assert count == 2

__보존할 상태를 캡슐화하는 작은 클래스를 정의하는 것이다.__

In [9]:
class CountMissing(object):
    def __init__(self):
        self.added = 0
        
    def missing(self):
        self.added += 1
        return 0

Python은 `first-class object` 덕분에 객체로 CountMissing.missing method를 직접 참조해서 defaultdict의 기본값 후크로 넘길 수 있다.

In [10]:
counter = CountMissing()
result = defaultdict(counter.missing, current) # method 참조

for key, amount in increments:
    result[key] += amount
assert counter.added == 2

핼퍼 클래스로 상태 보존 closure의 동작을 제공하는 방법이 명확하다. 그러나 CountMissing 클래스 자체만으로는 용도를 파악하기 어렵다.

Python에서는 클래스에 `__call__`이라는 특별한 메서드를 정의한다.  
`__call__` method는 객체를 함수처럼 호출할 수 있게 해준다. 또한 내장 함수 callable이 이런 인스턴스에 대해서는 True를 반환하게 만든다.

In [12]:
class BetterCountMissing(object):
    def __init__(self):
        self.added = 0
        
    def __call__(self):
        self.added += 1
        return 0
    
counter = BetterCountMissing()
counter()
assert callable(counter)

다음은 BetterCountMissing 인스턴스를 defaultdict의 기본값 후크로 사용하여 dictionary에 없어서 새로 추가된 키의 개수를 알아내는 코드다.

In [15]:
counter = BetterCountMissing()
result = defaultdict(counter, current) # __call__이 필요함
for key, amount in increments:
    result[key] += amount
    
assert counter.added == 2

`__call__` method는 함수 인수를 사용하기 적합한 위치에 클래스의 인스턴스를 사용할 수 있다는 사실을 드러낸다.  
__이 코드를 처음 보는 사람을 클래스의 주요 동작을 책임지는 진입점(entry point)으로 안내하는 역할도 한다.__  
클래스의 목적이 상태 보존 closure로 동작하는 것이라는 강력한 힌트를 제공한다.

`__call__`을 사용할 때 defaultdict는 여전히 무슨 일이 일어나는지 모른다. defaultdict에 필요한 건 후크용 함수 뿐이다. Python은 하고자 하는 작업에 따라 간단한 함수 인터페이스를 충족하는 다양한 방법을 제공한다.